In [26]:
import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold


import boto3

In [27]:
module_path = os.path.abspath(os.path.join('../../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [28]:
from global_config import ROOT_DIR, emotion_id_to_emotion_abr, conf_cmap
from src.analysis.supervised_learning.evaluation.confusion_matrix import ConfusionMatrixCreator

from src.preprocessing.dataset_creation.aggregation import get_aggregate_measures

In [48]:
def evaluate_scores(X, y, svc, scoring_method):
    
    skf = KFold(n_splits=100, shuffle=True)
    
    # get scores
    scores = cross_validate(X=X, y=y,
                            estimator           = svc,
                            scoring             = [scoring_method],
                            verbose             = 1,
                            cv                  = skf.split(X, y),
                            n_jobs              = -1,
                            return_train_score  = True                        
                           )
    
    print('printing {} measures'.format(scoring_method))
    print('avg (train):', np.mean(scores['train_{}'.format(scoring_method)]))
    print('std (train):', np.std(scores['train_{}'.format(scoring_method)]))
    print('avg (validation):', np.mean(scores['test_{}'.format(scoring_method)]))
    print('std (validation):', np.std(scores['test_{}'.format(scoring_method)]))

In [30]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()
from sagemaker import get_execution_role

# this datafile has standardscaled data on a per actor basis
role = get_execution_role()
bucket='files-and-examples-01'
file = 'datasets/su_dataset/all_data.pickle'

path = s3.open('s3://{}/{}'.format(bucket, file))

In [31]:
file = pickle.load(path)

x = file['x']
y = file['y']

au = x['au']
gaze = x['gaze']
pose = x['pose']

In [32]:
len(au)

10963

In [33]:
au[0].shape

(38, 17)

In [34]:
au[1].shape

(69, 17)

In [35]:
au_agg = get_aggregate_measures(au,
                               means=True,
                               variance=False,
                               deltas=False,
                               peaks=False)

In [36]:
scaler = MinMaxScaler()
au_agg = scaler.fit_transform(au_agg)

In [37]:
skf = KFold(n_splits=5, shuffle=True)
splits = skf.split(au_agg, y)

In [38]:
# regularization paramater, lower C -> more regularization (less overfitting), large C -> less regularization (and possibly more overfitting)
c_values = [0.1, 1, 5, 10, 25, 50, 75, 100]

# 
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
kernel = ['rbf', 'linear', 'poly', 'sigmoid']

parameters = {'class_weight': ['balanced'],
              'C': c_values,
              'gamma': gamma,
              'kernel': kernel,
              }

svc = SVC()
clf = GridSearchCV(estimator=svc,
                   param_grid=parameters,
                   scoring='accuracy',
                   verbose=5,
                   cv=splits,
                   n_jobs=-1,
                   )

In [39]:
clf.fit(au_agg, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  5.1min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x7fe6a49447d0>,
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10, 25, 50, 75, 100],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [40]:
clf.best_params_

{'C': 10, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}

In [49]:
svc = SVC(**clf.best_params_)
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 out of 100 | elapsed:  1.4min remaining:   59.9s


printing accuracy measures
avg (train): 0.9549715918192548
std (train): 0.0005959840092633306
avg (validation): 0.23423603002502083
std (validation): 0.04235634749554646


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.1min finished


# Grid Search without shuffle/min max normalization

In [16]:
clf.best_params_

{'C': 1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'rbf'}

In [17]:
svc = SVC(**clf.best_params_)
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


printing accuracy measures
avg (train): 0.19410736662373554
std (train): 0.004637956432709442
avg (validation): 0.12341420611700801
std (validation): 0.0124849801140342


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   26.1s finished


In [21]:
#**{'C': 10, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}

svc = SVC(**{'C': 2, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'rbf'})
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


printing accuracy measures
avg (train): 0.23225845460713476
std (train): 0.004177855646772091
avg (validation): 0.12095182581605039
std (validation): 0.009047825781993136


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.1s finished


In [23]:
#scaler = MinMaxScaler()
#au_agg = scaler.fit_transform(au_agg)
au_agg, y = shuffle(au_agg, y)

In [24]:
svc = SVC(**clf.best_params_)
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


printing accuracy measures
avg (train): 0.1940390997751349
std (train): 0.002745758375877358
avg (validation): 0.13746380320928236
std (validation): 0.011494319533980748


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.9s finished
